<a href="https://colab.research.google.com/github/LauraOrozco/NLP_course/blob/main/Taller2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Universidad del Valle**

**Maestría en Analítica e Inteligencia de Negocios**

**Procesamiento de lenguaje natural**

**Orozco Laura**

**Santa Kellys**

**Sanchez Jose**

In [1]:
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
files_folder = "/content/drive/MyDrive/Maestría analítica/Tercer Semestre/PNL/"

In [4]:
def leer_archivo(ruta):
  with open(ruta,  mode='r',encoding='utf-8') as f:
    texto = f.read()
  return texto

In [5]:
def limpiar_sentencias(sentencias):
  sentencias_nuevas = []
  regexs = [
    re.compile(p)
    for p in [r'https?://\S+', # Quitar URLs
              '[&#@]',  # Quitar los simbolos (&,#, @)
              r':-*\(+',  # Quitar simbolos :(  :-(
              r':-*\)+',  # Quitar simbolos :)  :-)
              '[:,\?\.\!\-\_]+',  # Quitar otros simbolos
              ]
        ]
  #Quitar emoticones
  emoticones = re.compile(
        '['
        u'\U0001F600-\U0001F64F' 
        u'\U0001F300-\U0001F5FF'
        u'\U0001F680-\U0001F6FF' 
        u'\U0001F1E0-\U0001F1FF'
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
  
  #Quitar emoticones
  sentencias_nuevas = [emoticones.sub('', s) for s in sentencias]

  #Quitar espacios en blanco
  espacios_blanco = re.compile(' +')

  for regex in regexs:
    print("quitando",regex)
    sentencias_nuevas = [regex.sub('', s) for s in sentencias_nuevas]
  sentencias_nuevas = [espacios_blanco.sub(' ', s) for s in sentencias_nuevas]
  return sentencias_nuevas

In [6]:
def segmentar_texto(texto):
  ##Segmentar el texto
  sentencias = re.split(r'[\n]+', texto)
  return sentencias

In [8]:
import os
os.getcwd()

'/content'

In [10]:
texto = leer_archivo("/content/drive/MyDrive/Maestría analítica/Tercer Semestre/PNL/gabriel_garcia_marquez_cien_annos_soledad.txt")
sentencias_texto = segmentar_texto(texto)
sentencias_limpias = limpiar_sentencias(sentencias_texto)

quitando re.compile('https?://\\S+')
quitando re.compile('[&#@]')
quitando re.compile(':-*\\(+')
quitando re.compile(':-*\\)+')
quitando re.compile('[:,\\?\\.\\!\\-\\_]+')


In [12]:
import os
import sys
import io
import multiprocessing
import pickle
from gensim.models import Word2Vec
import json
from nltk import sent_tokenize, word_tokenize
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

folder = '/content/drive/MyDrive/Maestría analítica/Tercer Semestre/PNL/'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


0it [00:00, ?it/s]

In [15]:
import nltk

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
all_sentences = []
#f = open(folder + "ancora_all.txt", "rb")
num_lines = sum(1 for line in open("/content/drive/MyDrive/Maestría analítica/Tercer Semestre/PNL/gabriel_garcia_marquez_cien_annos_soledad.txt",'r'))

f = io.open("/content/drive/MyDrive/Maestría analítica/Tercer Semestre/PNL/gabriel_garcia_marquez_cien_annos_soledad.txt", 'r', encoding='utf8')

for x in tqdm(f, total=num_lines):
    tokens = word_tokenize(x, "spanish")
    if len(tokens) != 0:
        if x[:4] != '<doc' and x[:5] != '</doc':
            all_sentences.append(tokens)

print('No. Oraciones en total dataset: ', len(all_sentences), '\n')

print(all_sentences[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/11929 [00:00<?, ?it/s]

No. Oraciones en total dataset:  9503 

['Gabriel', 'García', 'Márquez']


In [18]:
sentences = []
diccionario = dict()

num_lines2 = len(all_sentences)

for i, sent in tqdm(enumerate(all_sentences), total=num_lines2):
    sente = []
    for k, word in enumerate(sent):
        sente.append(word)
        if word not in diccionario:
            diccionario[word] = 1
        else:
            diccionario[word] += 1

    sentences.append(sente)
        
print(sentences[1:5])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/9503 [00:00<?, ?it/s]

[['Cien', 'años', 'de', 'soledad'], ['EDITADO', 'POR', '``', 'EDICIONES', 'LA', 'CUEVA', "''"], ['Para', 'J', 'omi', 'García', 'Ascot'], ['y', 'María', 'Luisa', 'Elio']]


In [19]:
print ("numero de oraciones presentes en el corpus: " + str(len(sentences)))
print ("                 numero de palabras unicas: " + str(len(diccionario)))

numero de oraciones presentes en el corpus: 9503
                 numero de palabras unicas: 17009


In [21]:
from gensim.models import Word2Vec
num_features = 300                      #Dimensionality of the resulting word vectors
min_word_count = 1                            #Minimum word count threshold
num_workers = multiprocessing.cpu_count()     #Number of threads to run in parallel
context_size = 8                              #Context window length
seed = 1                                      #Seed for the RNG, to make the result reproducible
model_1 = Word2Vec(sentences=sentences, size=100, window=5, min_count=1, workers=4)
model_2 = Word2Vec(sentences=sentences, size=300, window=5, min_count=1, workers=4)